In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("df_og.xlsx")

In [3]:
df.shape

(100514, 17)

In [4]:
missing = df.isnull()
missing.sum()

Unnamed: 0                       0
Loan Status                    514
Current Loan Amount            514
Term                           514
Credit Score                 19668
Annual Income                19668
Years in current job          4736
Home Ownership                 514
Purpose                        514
Monthly Debt                   514
Years of Credit History        514
Number of Open Accounts        514
Number of Credit Problems      514
Current Credit Balance         514
Maximum Open Credit            516
Bankruptcies                   718
Tax Liens                      524
dtype: int64

In [5]:
df_full = df

In [6]:
df_full = df_full.dropna(subset=['Loan Status'])

In [7]:
df_full.isnull().sum()

Unnamed: 0                       0
Loan Status                      0
Current Loan Amount              0
Term                             0
Credit Score                 19154
Annual Income                19154
Years in current job          4222
Home Ownership                   0
Purpose                          0
Monthly Debt                     0
Years of Credit History          0
Number of Open Accounts          0
Number of Credit Problems        0
Current Credit Balance           0
Maximum Open Credit              2
Bankruptcies                   204
Tax Liens                       10
dtype: int64

In [8]:
df_full = df_full.dropna(subset=['Loan Status'])
df_full2 = df_full.dropna(subset=['Maximum Open Credit'])
df_full2 = df_full2.dropna(subset=['Bankruptcies'])
df_full2 = df_full2.dropna(subset=['Tax Liens'])
df_full2 = df_full2.drop(['Unnamed: 0'], axis=1)
df_full2.shape

(99794, 16)

In [9]:
df_full2.isnull().sum()

Loan Status                      0
Current Loan Amount              0
Term                             0
Credit Score                 19111
Annual Income                19111
Years in current job          4222
Home Ownership                   0
Purpose                          0
Monthly Debt                     0
Years of Credit History          0
Number of Open Accounts          0
Number of Credit Problems        0
Current Credit Balance           0
Maximum Open Credit              0
Bankruptcies                     0
Tax Liens                        0
dtype: int64

In [10]:
#export data
df_OG = pd.DataFrame(data = df_full2)
df.to_csv('df_OG.csv', index=False, header=True)

In [11]:
df2 = pd.read_csv("df_OG.csv")

In [13]:
term = []
#short term = 0, long term = 1
for t in df2['Term']: 
    if t == 'Short Term': 
        t = 0
        term.append(t)
    else: 
        t = 1 
        term.append(t)

df2.drop(['Term'], axis = 1, inplace = True)
df2['Term'] = term

In [27]:
df2.head()

,Loan Status,Current Loan Amount,Credit Score,Annual Income,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term,Years in Current Job
0,Fully Paid,445412.0,709.0,1167493.0,Home Mortgage,Home Improvements,5214.74,17.2,6.0,1.0,228190.0,416746.0,1.0,0.0,0,8
1,Fully Paid,262328.0,NaN,NaN,Home Mortgage,Debt Consolidation,33295.98,21.1,35.0,0.0,229976.0,850784.0,0.0,0.0,0,10
2,Fully Paid,99999999.0,741.0,2231892.0,Own Home,Debt Consolidation,29200.53,14.9,18.0,1.0,297996.0,750090.0,0.0,0.0,0,8
3,Fully Paid,347666.0,721.0,806949.0,Own Home,Debt Consolidation,8741.90,12.0,9.0,0.0,256329.0,386958.0,0.0,0.0,1,3
4,Fully Paid,176220.0,NaN,NaN,Rent,Debt Consolidation,20639.70,6.1,15.0,0.0,253460.0,427174.0,0.0,0.0,0,5


In [15]:
df2.drop(['Unnamed: 0'], axis=1, inplace = True)

In [16]:
df2['Years in current job'].dtype

dtype('O')

In [24]:
df2['Years in current job'] = df2['Years in current job'].apply(lambda x: str(x))

In [26]:
#Years of Employment - major factor in deciding financial stability and secure income source
yearsinemployment = []

for years in df2['Years in current job']:
    y = ''.join(filter(lambda x : x.isdigit(), years))
    yearsinemployment.append(y)

#yearsinemployment
df2.drop(['Years in current job'], axis=1, inplace = True)
df2['Years in Current Job'] = yearsinemployment

In [46]:
#credit range 350-830 avg
newcreditscore = []
for credit in df2['Credit Score']: 
    if credit > 800:
        newcreditscore.append(credit/10)
    else: 
        newcreditscore.append(credit)

df2.drop(['Credit Score'], axis = 1 , inplace = True)
df2['Credit_Score'] = newcreditscore


In [48]:
df2.drop(['Credit_Score'], axis = 1 , inplace = True)

In [49]:
df2['Credit Score'] = newcreditscore

In [50]:
#bin credit score (source: experian)
#https://www.experian.com/blogs/ask-experian/credit-education/score-basics/what-is-a-good-credit-score/
#bins - 300-579, 580-669, 670-739, 740-799, 800-850
bins = [300, 580, 670, 740, 800, 850]
group_names = ['Very Poor', 'Fair', 'Good', 'Very Good', 'Exceptional']

df2['Credit Rating'] = pd.cut(df2['Credit Score'], bins, labels=group_names)
df2.head()

,Loan Status,Current Loan Amount,Annual Income,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term,Years in Current Job,Credit Rating,Credit Score
0,Fully Paid,445412.0,1167493.0,Home Mortgage,Home Improvements,5214.74,17.2,6.0,1.0,228190.0,416746.0,1.0,0.0,0,8,Good,709.0
1,Fully Paid,262328.0,NaN,Home Mortgage,Debt Consolidation,33295.98,21.1,35.0,0.0,229976.0,850784.0,0.0,0.0,0,10,NaN,NaN
2,Fully Paid,99999999.0,2231892.0,Own Home,Debt Consolidation,29200.53,14.9,18.0,1.0,297996.0,750090.0,0.0,0.0,0,8,Very Good,741.0
3,Fully Paid,347666.0,806949.0,Own Home,Debt Consolidation,8741.90,12.0,9.0,0.0,256329.0,386958.0,0.0,0.0,1,3,Good,721.0
4,Fully Paid,176220.0,NaN,Rent,Debt Consolidation,20639.70,6.1,15.0,0.0,253460.0,427174.0,0.0,0.0,0,5,NaN,NaN


In [51]:
#Encode loan status (fully paid = 0, charged off = 1)
loan_status = []

for status in df2['Loan Status']: 
    if status == 'Fully Paid': 
        status = 0
        loan_status.append(status)
    else: 
        status = 1
        loan_status.append(status)
        
df2['LoanStatus'] = loan_status

In [21]:
df2['Term'].dtype

dtype('int64')

In [22]:
df2['Term'] = df2['Term'].apply(lambda x: str(x))

In [23]:
#TERM 
#short term = 0, long term = 1
df2['Term'].replace(("Short Term","Long Term"),(0,1), inplace=True)
df2.head()

,Loan Status,Current Loan Amount,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term
0,Fully Paid,445412.0,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,6.0,1.0,228190.0,416746.0,1.0,0.0,0
1,Fully Paid,262328.0,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,35.0,0.0,229976.0,850784.0,0.0,0.0,0
2,Fully Paid,99999999.0,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,18.0,1.0,297996.0,750090.0,0.0,0.0,0
3,Fully Paid,347666.0,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,9.0,0.0,256329.0,386958.0,0.0,0.0,1
4,Fully Paid,176220.0,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,15.0,0.0,253460.0,427174.0,0.0,0.0,0


In [41]:
df2.shape

(100514, 16)

In [56]:
#Fill in missing annual income
df2['Annual Income'].fillna(df2['Annual Income'].mean(), inplace=True)

In [59]:
#Fill in missing credit score 
scount = []
lcount = []

for t in df2['Term']: 
    if t == '0': 
        scount.append(t)
    else: 
        t == 1 
        lcount.append(t)

print(len(scount))
print(len(lcount))


72208
28306


In [61]:
s_scoredf = df2[df2['Term']=='0']
#s_scoredf.head()
stermAVG = s_scoredf['Credit Score'].mean()
print(stermAVG)

724.6209386906824


In [64]:
l_scoredf = df2[df2['Term']=='1']
#l_scoredf.head()
ltermAVG = l_scoredf['Credit Score'].mean()
print(ltermAVG)

695.483727170432


In [79]:
df2.loc[(df2['Term'] =='0') & (df2['Credit Score'].isnull()), 'Credit Score'] = stermAVG

In [81]:
df2.loc[(df2['Term'] =='1') & (df2['Credit Score'].isnull()), 'Credit Score'] = ltermAVG

In [80]:
df2.head()

,Loan Status,Current Loan Amount,Annual Income,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term,Years in Current Job,Credit Rating,LoanStatus,Credit Score
0,Fully Paid,445412.0,1.167493e+06,Home Mortgage,Home Improvements,5214.74,17.2,6.0,1.0,228190.0,416746.0,1.0,0.0,0,8,Good,0,709.000000
1,Fully Paid,262328.0,1.378277e+06,Home Mortgage,Debt Consolidation,33295.98,21.1,35.0,0.0,229976.0,850784.0,0.0,0.0,0,10,NaN,0,724.620939
2,Fully Paid,99999999.0,2.231892e+06,Own Home,Debt Consolidation,29200.53,14.9,18.0,1.0,297996.0,750090.0,0.0,0.0,0,8,Very Good,0,741.000000
3,Fully Paid,347666.0,8.069490e+05,Own Home,Debt Consolidation,8741.90,12.0,9.0,0.0,256329.0,386958.0,0.0,0.0,1,3,Good,0,721.000000
4,Fully Paid,176220.0,1.378277e+06,Rent,Debt Consolidation,20639.70,6.1,15.0,0.0,253460.0,427174.0,0.0,0.0,0,5,NaN,0,724.620939


In [82]:
#credit range 350-830 avg
newcreditscore = []
for credit in df2['Credit Score']: 
    if credit > 800:
        newcreditscore.append(credit/10)
    else: 
        newcreditscore.append(credit)

df2.drop(['Credit Score'], axis = 1 , inplace = True)
df2['Credit Score'] = newcreditscore


In [83]:
#bin credit score (source: experian)
#https://www.experian.com/blogs/ask-experian/credit-education/score-basics/what-is-a-good-credit-score/
#bins - 300-579, 580-669, 670-739, 740-799, 800-850
bins = [300, 580, 670, 740, 800, 850]
group_names = ['Very Poor', 'Fair', 'Good', 'Very Good', 'Exceptional']

df2['Credit Rating'] = pd.cut(df2['Credit Score'], bins, labels=group_names)
df2.head()

,Loan Status,Current Loan Amount,Annual Income,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term,Years in Current Job,Credit Rating,LoanStatus,Credit Score
0,Fully Paid,445412.0,1.167493e+06,Home Mortgage,Home Improvements,5214.74,17.2,6.0,1.0,228190.0,416746.0,1.0,0.0,0,8,Good,0,709.000000
1,Fully Paid,262328.0,1.378277e+06,Home Mortgage,Debt Consolidation,33295.98,21.1,35.0,0.0,229976.0,850784.0,0.0,0.0,0,10,Good,0,724.620939
2,Fully Paid,99999999.0,2.231892e+06,Own Home,Debt Consolidation,29200.53,14.9,18.0,1.0,297996.0,750090.0,0.0,0.0,0,8,Very Good,0,741.000000
3,Fully Paid,347666.0,8.069490e+05,Own Home,Debt Consolidation,8741.90,12.0,9.0,0.0,256329.0,386958.0,0.0,0.0,1,3,Good,0,721.000000
4,Fully Paid,176220.0,1.378277e+06,Rent,Debt Consolidation,20639.70,6.1,15.0,0.0,253460.0,427174.0,0.0,0.0,0,5,Good,0,724.620939


In [87]:
#monthly income
monthlyincome = []
for income in df2['Annual Income']: 
    monthlyincome.append(income/12)

#df2.drop(['Credit Score'], axis = 1 , inplace = True)
df2['Monthly Income'] = monthlyincome

In [91]:
df2

,Loan Status,Current Loan Amount,Annual Income,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term,Years in Current Job,Credit Rating,LoanStatus,Credit Score,Monthly Income
0,Fully Paid,445412.0,1.167493e+06,Home Mortgage,Home Improvements,5214.74,17.2,6.0,1.0,228190.0,416746.0,1.0,0.0,0,8,Good,0,709.000000,97291.083333
1,Fully Paid,262328.0,1.378277e+06,Home Mortgage,Debt Consolidation,33295.98,21.1,35.0,0.0,229976.0,850784.0,0.0,0.0,0,10,Good,0,724.620939,114856.379987
2,Fully Paid,99999999.0,2.231892e+06,Own Home,Debt Consolidation,29200.53,14.9,18.0,1.0,297996.0,750090.0,0.0,0.0,0,8,Very Good,0,741.000000,185991.000000
3,Fully Paid,347666.0,8.069490e+05,Own Home,Debt Consolidation,8741.90,12.0,9.0,0.0,256329.0,386958.0,0.0,0.0,1,3,Good,0,721.000000,67245.750000
4,Fully Paid,176220.0,1.378277e+06,Rent,Debt Consolidation,20639.70,6.1,15.0,0.0,253460.0,427174.0,0.0,0.0,0,5,Good,0,724.620939,114856.379987
5,Charged Off,206602.0,8.968570e+05,Home Mortgage,Debt Consolidation,16367.74,17.3,6.0,0.0,215308.0,272448.0,0.0,0.0,0,10,Good,1,729.000000,74738.083333
6,Fully Paid,217646.0,1.184194e+06,Home Mortgage,Debt Consolidation,10855.08,19.6,13.0,1.0,122170.0,272052.0,1.0,0.0,0,1,Good,0,730.000000,98682.833333
7,Charged Off,648714.0,1.378277e+06,Home Mortgage,Buy House,14806.13,8.2,15.0,0.0,193306.0,864204.0,0.0,0.0,1,1,Good,1,695.483727,114856.379987
8,Fully Paid,548746.0,2.559110e+06,Rent,Debt Consolidation,18660.28,22.6,4.0,0.0,437171.0,555038.0,0.0,0.0,0,2,Good,0,678.000000,213259.166667
9,Fully Paid,215952.0,1.454735e+06,Rent,Debt Consolidation,39277.75,13.9,20.0,0.0,669560.0,1021460.0,0.0,0.0,0,1,Good,0,739.000000,121227.916667


In [90]:
#export dataframe

df_CLEAN = pd.DataFrame(data = df2)
df.to_csv('df_CLEAN.csv', index=False, header=True)

In [92]:
df2.isnull().sum()

Loan Status                  514
Current Loan Amount          514
Annual Income                  0
Home Ownership               514
Purpose                      514
Monthly Debt                 514
Years of Credit History      514
Number of Open Accounts      514
Number of Credit Problems    514
Current Credit Balance       514
Maximum Open Credit          516
Bankruptcies                 718
Tax Liens                    524
Term                           0
Years in Current Job           0
Credit Rating                  0
LoanStatus                     0
Credit Score                   0
Monthly Income                 0
dtype: int64

In [94]:
df_full = df2.dropna(subset=['Loan Status'])
df_full = df_full.dropna(subset=['Maximum Open Credit'])
df_full = df_full.dropna(subset=['Bankruptcies'])
df_full = df_full.dropna(subset=['Tax Liens'])
#df_full = df_full.drop(['Unnamed: 0'], axis=1)
df_full.shape

(99794, 19)

In [96]:
df_full.isnull().sum()

Loan Status                  0
Current Loan Amount          0
Annual Income                0
Home Ownership               0
Purpose                      0
Monthly Debt                 0
Years of Credit History      0
Number of Open Accounts      0
Number of Credit Problems    0
Current Credit Balance       0
Maximum Open Credit          0
Bankruptcies                 0
Tax Liens                    0
Term                         0
Years in Current Job         0
Credit Rating                0
LoanStatus                   0
Credit Score                 0
Monthly Income               0
dtype: int64

In [100]:
#export dataframe

df_plzCLEAN = pd.DataFrame(data = df_full)
df_plzCLEAN.to_csv('df_plzCLEAN.csv', index=False, header=True)

In [111]:
df_full.corr()

,Current Loan Amount,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,LoanStatus,Credit Score,Monthly Income
Current Loan Amount,1.000000,0.013078,-0.006571,0.019303,0.001656,-0.002834,0.003992,-0.001126,-0.000611,-0.002039,-0.194571,0.126695,0.013078
Annual Income,0.013078,1.000000,0.438551,0.144903,0.131580,-0.015489,0.284287,0.038990,-0.043115,0.037079,-0.046703,-0.005007,1.000000
Monthly Debt,-0.006571,0.438551,1.000000,0.199743,0.411185,-0.055744,0.481000,0.039532,-0.078977,0.020025,0.008023,-0.092853,0.438551
Years of Credit History,0.019303,0.144903,0.199743,1.000000,0.132226,0.061887,0.208432,0.030742,0.066248,0.017345,-0.023650,0.064503,0.144903
Number of Open Accounts,0.001656,0.131580,0.411185,0.132226,1.000000,-0.014184,0.227776,0.030985,-0.024582,0.006499,0.012166,-0.032783,0.131580
Number of Credit Problems,-0.002834,-0.015489,-0.055744,0.061887,-0.014184,1.000000,-0.112704,-0.012011,0.752941,0.581352,0.002347,-0.040145,-0.015489
Current Credit Balance,0.003992,0.284287,0.481000,0.208432,0.227776,-0.112704,1.000000,0.138464,-0.122609,-0.015677,-0.009332,-0.023049,0.284287
Maximum Open Credit,-0.001126,0.038990,0.039532,0.030742,0.030985,-0.012011,0.138464,1.000000,-0.014574,-0.001022,-0.008215,0.011782,0.038990
Bankruptcies,-0.000611,-0.043115,-0.078977,0.066248,-0.024582,0.752941,-0.122609,-0.014574,1.000000,0.046109,-0.006572,-0.028765,-0.043115
Tax Liens,-0.002039,0.037079,0.020025,0.017345,0.006499,0.581352,-0.015677,-0.001022,0.046109,1.000000,0.010249,-0.018498,0.037079


In [107]:
df_full['Years in Current Job'].dtype

dtype('O')

In [110]:
df_full['Years in Current Job'] = df_full['Years in Current Job'].apply(lambda x: float(x))

ValueError: could not convert string to float: 

In [108]:

df_full['Years in Current Job'].describe()

count     99794
unique       11
top          10
freq      31090
Name: Years in Current Job, dtype: object